In [26]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math
from datetime import datetime

In [2]:
%matplotlib inline

In [3]:
plt.rcParams['figure.figsize'] = (10, 5)

In [4]:
saved_style_state = matplotlib.rcParams.copy()

In [6]:
if os.path.isfile("Cycleway Counts_01Jan18_11Sept20.xlsx"):
    filepath = "Cycleway Counts_01Jan18_11Sept20.xlsx"
    print("loading from file")
else:
    filepath = "https://opendata.transport.nsw.gov.au/system/files/resources/Cycleway%20Counts_01Jan18_11Sept20.xlsx"
    print("loading from the internet")

cycleway_usage = pd.read_excel(filepath, sheet_name="daily counts")
print("done")


loading from file
done


In [7]:
cycleway_usage.head()

,Average Daily Cycle Count,"100005 - Merremburn Avenue, Naremburn (offramp path) (Cycleway and Pedestrian counter)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 486,Unnamed: 487,Unnamed: 488,Unnamed: 489,Unnamed: 490,Unnamed: 491,Unnamed: 492,Unnamed: 493,Unnamed: 494,Unnamed: 495
0,NaN,Cycling,NaN,NaN,NaN,NaN,Pedestrian,NaN,NaN,NaN,...,Pedestrian,NaN,NaN,NaN,NaN,Sum (All Transport Modes),NaN,NaN,NaN,NaN
1,NaN,North,East,South,West,All Directions,North,East,South,West,...,North,East,South,West,All Directions,North,East,South,West,All Directions
2,01/01/2018,,,,,,,,,,...,85.666667,50.833333,85.5,50.666667,272.666667,161,85.566667,153.966667,83.866667,484.4
3,02/01/2018,,,,,,,,,,...,71,44.333333,71.166667,44.333333,230.833333,192.2,102.733333,188.7,100.133333,583.766667
4,03/01/2018,,,,,,,,,,...,73,48.166667,73.166667,48.166667,242.5,195.466667,103.433333,189.766667,102.433333,591.1


In [23]:
dir_lookup = ["North", "East", "South", "West", "All Directions"]
def get_mode(index):
    if index<5:
        return "Cycling"
    else:
        return "Pedestrian"

locations_sparse = [x for x in cycleway_usage.columns if "Unnamed" not in x ]

In [36]:
def get_loc(i, response_type="all"):
    try:
        loc = locations_sparse[math.ceil(i/15)]
        if response_type == "all":
            return loc
        elif response_type == "code":
            return loc.split("-")[0].strip()
        elif response_type == "name":
            return loc.split("-")[1].strip()
        # You can extend this to pull out the notes, like (Cycleway and Pedestrian counter), but they're pretty inconsistent
        else:
            print(f"`{response_type}` is not implemented yet")
    except:
        print("er")


In [41]:
new_rows = []
for i, row in cycleway_usage.iterrows():
    if i < 3:
        continue  # This feels nasty

    date = row[0]
    for loc_index in range(1, len(row) - 15, 15):
        section = row[
            loc_index : loc_index + 10
        ]  # 10 because we don't care about the Sum (All Transport Modes) section
        for col_index, col in enumerate(section):
            dir = dir_lookup[col_index % 5]
            if type(col) is int and col != 0 and dir != "All Directions":
                r = {
                    "date": date,
                    "location_name": get_loc(loc_index, "name"),
                    "location_code": get_loc(loc_index, "code"),
                    "mode": get_mode(col_index),
                    "direction": dir,
                    "count": int(col),
                }
                new_rows.append(r)


In [42]:
t_df = pd.DataFrame(new_rows)  # make a working DF


In [44]:
print(t_df.shape)
t_df.head()

(43329, 6)


,date,location_name,location_code,mode,direction,count
0,02/01/2018,"Falcon St Pedestrian Bridge , Cammeray (Cyclew...",100008,Cycling,North,9
1,02/01/2018,"Falcon St Pedestrian Bridge , Cammeray (Cyclew...",100008,Cycling,South,5
2,02/01/2018,"Falcon St Pedestrian Bridge , Cammeray (Cyclew...",100008,Pedestrian,North,7
3,02/01/2018,"Falcon St Pedestrian Bridge , Cammeray (Cyclew...",100008,Pedestrian,South,7
4,02/01/2018,"Ridge St Bridge, North Sydney (Cycleway and Pe...",100009,Pedestrian,East,88


# What's left to do here?

* read through my code and work out how it works, it's a bit grotty, and brittle in places
* Split the notes off the location name, and put them into their own column
* convert the date column to an actual date, but maybe do that as part of the import
* perhaps sort the DF by date

but you can do that as you go with the analysis and you see what it all looks like